# Preprocessing

This notebook is used to process the raw transcripts into a format that can then be used for model training and testing.


In [1]:
#!/usr/bin/env/python
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import string
import unicodedata
import matplotlib
import matplotlib.pyplot as plt
import re

In [2]:
#Load Subset 1 and Subset 2
path1 = '../data/Subset 1/'
subdirs = glob.glob('%s*'%(path1))

path2 = '../data/Subset 2/'
subdirs.extend(glob.glob('%s*'%(path2)))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [3]:
def resolve_label(label):
    if label == '1':
        return '0'
    elif label =='2':
        return '1'
    elif label =='3':
        return '2'
    elif label =='4':
        return '3'
    elif label =='5':
        return '4'
    elif label =='8':
        return '5'
    elif label =='9':
        return '6'
    else:
        return 'nan'
    
def resolve_slabel(label):
    if label == '1':
        return '0'
    elif label =='2':
        return '1'
    elif label =='3':
        return '2'
    elif label =='4':
        return '3'
    elif label =='5':
        return '4'
    else:
        return 'nan'

In [4]:
'''
To avoid overfitting the models we carefully divide the training and testing set. 
Testing set include some teachers who particiapted in Year 1. The transcripts from these teachers are not included
in the training set. The teachers in this list were picked at random.
'''

train_subdirs =[]
test_subdirs = []
for item in subdirs:
    if item.find("Travers.Spring") != -1 or item.find("Benson") != -1 \
    or item.find("Carroll") != -1 or item.find("Saunders.Spring") != -1 \
    or item.find("Keene") != -1 or item.find("Carter") != -1 or item.find("Basker") != -1 :
        test_subdirs.append(item)
    else:
        train_subdirs.append(item)
print(len(train_subdirs), len(test_subdirs))

504 63


In [12]:
data = None
temp = []
flag = 0
for dir in train_subdirs: #replace with test_subdirs to generate test data
    if dir.endswith('.xlsx') and not dir.startswith('~'):
        try:
            temp_data = None
            load_data = pd.read_excel(dir, engine='openpyxl')
            #print(dir, load_data.columns)
            load_data['Sentence'] = load_data['Sentence'].str.replace("[\(\[].*?[\)\]]", "", regex=True)
            load_data = load_data[load_data['Sentence'] != ""]
            mask = load_data['Sentence'].str.contains('(.*[\(].*|.*[\)].*|.*[\[].*|.*[\]].*)|(^[.?!]+$)|(^.*[0-9]{1,2}:[0-9]{1,2}:[0-9]{1,2}.*$)',regex=True,na=False)
            temp_data = load_data[:][~mask]
            temp_data = temp_data.loc[:, ~temp_data.columns.str.contains('^Unnamed')]
            temp_data['Transcript'] = dir.split('/')[len(dir.split('/'))-1]
            for item in temp_data.columns:
                if item not in ['Teacher Tag', 'TimeStamp' ,'Turn','Student Tag','Sentence','Speaker','Transcript']:
                    temp_data = temp_data.drop(columns=[item])
            if flag == 0:
                data = temp_data
                flag = 1
            else:
                if len(data.columns) > 8:
                    print(data.columns,dir)
                data = data.append(temp_data, ignore_index=True,sort=False)
        except Exception as e:
            print(e,dir)
            pass 

/tmp/ipykernel_19919/3975454404.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mask = load_data['Sentence'].str.contains('(.*[\(].*|.*[\)].*|.*[\[].*|.*[\]].*)|(^[.?!]+$)|(^.*[0-9]{1,2}:[0-9]{1,2}:[0-9]{1,2}.*$)',regex=True,na=False)


In [13]:
len(data), len(subdirs)

(205313, 567)

In [14]:
dataset = []
bad_sentences = []
bad_speakers = []
num_sentences = []
bad_entries = []
translator = str.maketrans('', '', string.punctuation)
for entry in data.iterrows():
    try:
        if type(entry[1]['Speaker']) == int or entry[1]['Speaker'] == float:
            bad_speakers.append(entry[1]['Speaker'])
            continue
        if repr(type(entry[1]['Sentence'])) == "<type 'datetime.time'>" or repr(type(entry[1]['Sentence'])) == "<type 'datetime.datetime'>" or repr(entry[1]['Speaker']) == 'nan':
            bad_sentences.append(entry[1]['Sentence'])
            continue
    
        if repr(type(entry[1]['Sentence'])) == "<type 'int'>" or repr(type(entry[1]['Sentence'])) == "<type 'float'>":
            num_sentences.append(str(entry[1]['Sentence']))
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
        if repr(type(entry[1]['Sentence'])) == "<type 'str'>":
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
            
        temp = unicodedata.normalize('NFKD', str(entry[1]['Sentence']).translate(translator))
        
        # Turn, Speaker, Sentence, Tag
        try:
            tag = resolve_label(str(entry[1]['Teacher Tag']).split('-')[0].strip())
        except Exception as e:
            print(e,'####')
        try:
            stag = resolve_slabel(str(entry[1]['Student Tag']).split('-')[0].strip())
        except Exception as e:
            print(e,'####')
        
        sent = ' '.join([re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in temp.split(" ")])
        
        if str(stag) != 'nan':
            #print([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['Transcript']])
        elif str(tag) != 'nan':
            #print([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['Transcript']])
       
    except Exception as e:
        print(e)
        bad_entries.append([entry[1]['Sentence'], entry[1]['Speaker']])
        continue

In [15]:
df = pd.DataFrame({'Turn':[i[0] for i in dataset], 'Speaker':[i[1] for i in dataset], 'Sentence':[i[2] for i in dataset], 'Tag':[i[3] for i in dataset],'StudentTag':[i[4] for i in dataset],'Transcript':[i[5] for i in dataset]})
df = df.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df = df[df['Sentence'] != '']
df

,Turn,Speaker,Sentence,Tag,StudentTag,Transcript
0,1.0,T,My names Ms Villarin I dont know if Ms Ferrant...,0,nan,7th grade math.xlsx
1,1.0,T,So today her and I will be teaching a lesson t...,0,nan,7th grade math.xlsx
2,1.0,T,So you can treat me the way you treat her if y...,0,nan,7th grade math.xlsx
3,1.0,T,Okay,0,nan,7th grade math.xlsx
4,1.0,T,So today were going to be doing a really cool ...,0,nan,7th grade math.xlsx
...,...,...,...,...,...,...
203645,7,T,I sent it to you,0,nan,TalkBack.Year2.Wolfram.Fall.121720-3.xlsx
203646,8,S,All right thank you,nan,0,TalkBack.Year2.Wolfram.Fall.121720-3.xlsx
203647,9,T,But you should get in there and you actually d...,0,nan,TalkBack.Year2.Wolfram.Fall.121720-3.xlsx
203648,9,T,There you go,0,nan,TalkBack.Year2.Wolfram.Fall.121720-3.xlsx


In [ ]:
df[df['Sentence'] == '']

In [ ]:
df[df['Speaker']=='S']

In [ ]:
'''
  Number transcripts - 567
  Number of teacher utterances - 165,674
  Number of words (with repetitions) - 1,700,036
  Number of unique words - 15,749
  Number of student utterances - 56,580
  Number of Turns - 87,071
'''

In [ ]:
for item in sorted(df['Transcript'].unique()):
    print(item, len(df[df['Transcript']==item]))

In [ ]:
temp_df = df[df['Speaker'] == 'T']
temp_df['Tag'].value_counts(), temp_df['Tag'].value_counts().sum()


In [ ]:
df['word_count'] = df['Sentence'].apply(lambda x: len(str(x).split(' ')))

In [ ]:
df['word_count'].sum()

In [ ]:
numberOfTurns, prevTurn = 0, ''
for item in df.iterrows():
    if prevTurn == '': 
        numberOfTurns += 1
        prevTurn = str(item[1]['Turn'])
    if str(item[1]['Turn']) != str(prevTurn):
        numberOfTurns += 1
        prevTurn = str(item[1]['Turn'])
print(numberOfTurns)

In [ ]:
words = df['Sentence'].str.lower().str.findall("\w+")

unique = set()

for x in words:
    unique.update(x)
print(len(words),len(unique))

In [ ]:
df2 = df.loc[df['Speaker']=='T']
df2.groupby('Tag').agg(['count', 'size', 'nunique']).stack()

In [16]:
writer = pd.ExcelWriter('../data/train_data_504.xlsx') #replace with test_data_63 for test data
df.to_excel(writer)
writer.save()

In [ ]:
'''
Pre-processing 

- All lower cased
- All punctuation removed
- Converted all teachers to T tags
- Converted all students to S tags
- T labels range from  [0-6] - 0 stands for None
- S are labelled None
- Compiled all the data into single excel sheet
- At present there are 91891 sentences in total

Cleaning

- [End of Class], [Class Ends] - Sentence in brackets
- If Sentence contains a timestamp or date time stamp
- If sentence is just . or ! or ? or empty
- Speaker is a number (All of them are nans)
- Speaker is nan


'''

In [17]:
#Prepare Tuples from data - Student
data = pd.read_excel('../data/train_data_504.xlsx') #replace with test_data_63 to generate test data
empty_previous = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        flag = 1
        continue
    if row[1]['Speaker'] == 'S' and flag == 0:
        temp =  row[1]['Sentence']
        flag=1
        sentence_tuple.append([empty_previous,temp,row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        temp =  row[1]['Sentence']
        continue
        
df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple], 'text_b':[i[1] for i in sentence_tuple], 'labels':[i[2] for i in sentence_tuple]})
print(len(df),set(y))
df.to_csv('../data/train_student.tsv',sep='\t',index=False)   #replace with test_student to generate test data  

52683 {0.0, 1.0, 2.0, 3.0, 4.0}


In [18]:
#Prepare Tuples from data - teacher
data = pd.read_excel('../data/train_data_504.xlsx') #replace with test_data_63 to generate test data
empty_student = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'S' and flag == 2:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        temp =  row[1]['Sentence']
        flag = 2
        continue
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 2:
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        flag = 1
        continue

df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple], 'text_b':[i[1] for i in sentence_tuple], 'labels':[i[2] for i in sentence_tuple]})
print(len(df),set(y))
df.to_csv('../data/train_teacher.tsv',sep='\t',index=False)  #replace with test_teacher to generate test data 

150918 {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0}
